In [1]:
import json
import csv
import tweepy
import re
import nltk
from nltk.stem import WordNetLemmatizer
import pandas as pd
import GetOldTweets3 as got
from nltk.corpus import stopwords
stopwords = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
import numpy as np

In [2]:
vit1=pd.read_json('vit1.json')
vit2=pd.read_json('vit2.json')

In [3]:
tweets_df = pd.concat([vit1,vit2], axis=0)
tweets_df = tweets_df.loc[:, ['text' , 'username' ,'tweet_id', 'timestamp' ,'hashtags' , 'screen_name','retweets','likes']]
tweets_df

,text,username,tweet_id,timestamp,hashtags,screen_name,retweets,likes
0,Thanks to Developer Student Club-Vellore Insti...,Mohammad Shahid Ansari,1211639431959998464,2019-12-30 13:25:26,[],Shahid_Ansari_,0,0
1,Junior Research Fellow In Vellore Institute Of...,Sarkari-Job,1211536334608904192,2019-12-30 06:35:46,[governmentjob],sarkarijobind,0,0
2,VIT Notification 2020 – Opening for Various JR...,Govt Job Guide,1210822000945418240,2019-12-28 07:17:15,[],GovtJobGuide,0,0
3,"Vallore Institute of Technology (VIT) Courses,...",Rajasthan Travel and Tourism Guide,1210652629606354944,2019-12-27 20:04:14,"[rajasthanpalaces, royalrajasthan, rajasthanto...",e_rajasthan,0,0
4,MITS5509 Intelligent Systems for Analytics Ass...,Expertsmind.com,1209090557391208448,2019-12-23 12:37:07,"[MITS5509, IntelligentSystemsforAnalytics, Ass...",ExpertsMindInc,0,0
5,Thanks to Developer Student Club-Vellore Insti...,Mohammad Shahid Ansari,1211639431959998464,2019-12-30 13:25:26,[],Shahid_Ansari_,0,0
6,Junior Research Fellow In Vellore Institute Of...,Sarkari-Job,1211536334608904192,2019-12-30 06:35:46,[governmentjob],sarkarijobind,0,0
7,VIT Notification 2020 – Opening for Various JR...,Govt Job Guide,1210822000945418240,2019-12-28 07:17:15,[],GovtJobGuide,0,0
8,"Vallore Institute of Technology (VIT) Courses,...",Rajasthan Travel and Tourism Guide,1210652629606354944,2019-12-27 20:04:14,"[rajasthanpalaces, royalrajasthan, rajasthanto...",e_rajasthan,0,0
9,MITS5509 Intelligent Systems for Analytics Ass...,Expertsmind.com,1209090557391208448,2019-12-23 12:37:07,"[MITS5509, IntelligentSystemsforAnalytics, Ass...",ExpertsMindInc,0,0


In [4]:
tweets_df.reset_index(drop=True, inplace=True)

In [5]:
tweets_df = tweets_df[~tweets_df.index.duplicated()]
uni_tweets = tweets_df.loc[tweets_df['screen_name'].isin(['iGEM_VITVellore','ChennaiVit','VIT_univ'])]

In [6]:
uni_tweets['uni_handle'] = 1
uni_tweets

/Users/Manam/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,text,username,tweet_id,timestamp,hashtags,screen_name,retweets,likes,uni_handle
82,"""Founder and Chancellor of Vellore Institute o...",VIT University,1167289298598514688,2019-08-30 04:13:50,"[LanguageTeachingProgramme, ChineseLanguagetea...",VIT_univ,1,10,1
92,"""Founder and Chancellor of Vellore Institute o...",VIT University,1167289298598514688,2019-08-30 04:13:50,"[LanguageTeachingProgramme, ChineseLanguagetea...",VIT_univ,1,10,1
106,Vellore Institute of Technology and Binghamton...,VIT University,1145947328525684736,2019-07-02 06:48:28,"[VIT, VelloreInstituteofTechnology]",VIT_univ,1,8,1
118,Vellore Institute of Technology and Binghamton...,VIT University,1145947328525684736,2019-07-02 06:48:28,"[VIT, VelloreInstituteofTechnology]",VIT_univ,1,8,1
125,"The Vellore Institute of Technology (VIT), in ...",VIT University,1152105162162372608,2019-07-19 06:37:30,"[GlobalVIT, VIT, VelloreInstituteofTechnologypic]",VIT_univ,2,12,1
134,"The Vellore Institute of Technology (VIT), in ...",VIT University,1152105162162372608,2019-07-19 06:37:30,"[GlobalVIT, VIT, VelloreInstituteofTechnologypic]",VIT_univ,2,12,1
150,Vellore Institute of Technology wishes you a H...,VIT University,1161900914350837760,2019-08-15 07:22:19,"[VIT, VelloreInstituteofTechnology, Independen...",VIT_univ,0,16,1
161,Vellore Institute of Technology wishes you a H...,VIT University,1161900914350837760,2019-08-15 07:22:19,"[VIT, VelloreInstituteofTechnology, Independen...",VIT_univ,0,16,1
215,Vice Presidents of Vellore Institute of Techno...,VIT University,1171689619219349504,2019-09-11 07:39:08,"[TelanganaGovernor, VIT]",VIT_univ,0,13,1
231,Vice Presidents of Vellore Institute of Techno...,VIT University,1171689619219349504,2019-09-11 07:39:08,"[TelanganaGovernor, VIT]",VIT_univ,0,13,1


In [7]:
tweets_df = tweets_df[~tweets_df.index.duplicated()]
tweets_df['uni_handle'] = uni_tweets['uni_handle']
vit = tweets_df.combine_first(tweets_df)
vit

,text,username,tweet_id,timestamp,hashtags,screen_name,retweets,likes,uni_handle
0,Thanks to Developer Student Club-Vellore Insti...,Mohammad Shahid Ansari,1211639431959998464,2019-12-30 13:25:26,[],Shahid_Ansari_,0,0,NaN
1,Junior Research Fellow In Vellore Institute Of...,Sarkari-Job,1211536334608904192,2019-12-30 06:35:46,[governmentjob],sarkarijobind,0,0,NaN
2,VIT Notification 2020 – Opening for Various JR...,Govt Job Guide,1210822000945418240,2019-12-28 07:17:15,[],GovtJobGuide,0,0,NaN
3,"Vallore Institute of Technology (VIT) Courses,...",Rajasthan Travel and Tourism Guide,1210652629606354944,2019-12-27 20:04:14,"[rajasthanpalaces, royalrajasthan, rajasthanto...",e_rajasthan,0,0,NaN
4,MITS5509 Intelligent Systems for Analytics Ass...,Expertsmind.com,1209090557391208448,2019-12-23 12:37:07,"[MITS5509, IntelligentSystemsforAnalytics, Ass...",ExpertsMindInc,0,0,NaN
5,Thanks to Developer Student Club-Vellore Insti...,Mohammad Shahid Ansari,1211639431959998464,2019-12-30 13:25:26,[],Shahid_Ansari_,0,0,NaN
6,Junior Research Fellow In Vellore Institute Of...,Sarkari-Job,1211536334608904192,2019-12-30 06:35:46,[governmentjob],sarkarijobind,0,0,NaN
7,VIT Notification 2020 – Opening for Various JR...,Govt Job Guide,1210822000945418240,2019-12-28 07:17:15,[],GovtJobGuide,0,0,NaN
8,"Vallore Institute of Technology (VIT) Courses,...",Rajasthan Travel and Tourism Guide,1210652629606354944,2019-12-27 20:04:14,"[rajasthanpalaces, royalrajasthan, rajasthanto...",e_rajasthan,0,0,NaN
9,MITS5509 Intelligent Systems for Analytics Ass...,Expertsmind.com,1209090557391208448,2019-12-23 12:37:07,"[MITS5509, IntelligentSystemsforAnalytics, Ass...",ExpertsMindInc,0,0,NaN


In [8]:
vit['uni_handle'] = vit['uni_handle'].replace(np.nan, 0)
vit[vit['uni_handle'] == 1]

,text,username,tweet_id,timestamp,hashtags,screen_name,retweets,likes,uni_handle
82,"""Founder and Chancellor of Vellore Institute o...",VIT University,1167289298598514688,2019-08-30 04:13:50,"[LanguageTeachingProgramme, ChineseLanguagetea...",VIT_univ,1,10,1.0
92,"""Founder and Chancellor of Vellore Institute o...",VIT University,1167289298598514688,2019-08-30 04:13:50,"[LanguageTeachingProgramme, ChineseLanguagetea...",VIT_univ,1,10,1.0
106,Vellore Institute of Technology and Binghamton...,VIT University,1145947328525684736,2019-07-02 06:48:28,"[VIT, VelloreInstituteofTechnology]",VIT_univ,1,8,1.0
118,Vellore Institute of Technology and Binghamton...,VIT University,1145947328525684736,2019-07-02 06:48:28,"[VIT, VelloreInstituteofTechnology]",VIT_univ,1,8,1.0
125,"The Vellore Institute of Technology (VIT), in ...",VIT University,1152105162162372608,2019-07-19 06:37:30,"[GlobalVIT, VIT, VelloreInstituteofTechnologypic]",VIT_univ,2,12,1.0
134,"The Vellore Institute of Technology (VIT), in ...",VIT University,1152105162162372608,2019-07-19 06:37:30,"[GlobalVIT, VIT, VelloreInstituteofTechnologypic]",VIT_univ,2,12,1.0
150,Vellore Institute of Technology wishes you a H...,VIT University,1161900914350837760,2019-08-15 07:22:19,"[VIT, VelloreInstituteofTechnology, Independen...",VIT_univ,0,16,1.0
161,Vellore Institute of Technology wishes you a H...,VIT University,1161900914350837760,2019-08-15 07:22:19,"[VIT, VelloreInstituteofTechnology, Independen...",VIT_univ,0,16,1.0
215,Vice Presidents of Vellore Institute of Techno...,VIT University,1171689619219349504,2019-09-11 07:39:08,"[TelanganaGovernor, VIT]",VIT_univ,0,13,1.0
231,Vice Presidents of Vellore Institute of Techno...,VIT University,1171689619219349504,2019-09-11 07:39:08,"[TelanganaGovernor, VIT]",VIT_univ,0,13,1.0


In [9]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
vit['compound'] = [analyzer.polarity_scores(v)['compound'] for v in vit['text']]

In [10]:
def sentiment(compound):
    res = ""
    if(compound >= 0.05):
        res="Positive"
    elif(compound <= - 0.05):
        res="Negative"
    else:
        res="Neutral"
    return res

vit['Sentiment'] = vit['compound'].apply(lambda x: sentiment(x))

In [11]:
vit['score'] = pd.cut(vit['compound'], bins=5, labels=[1, 2, 3, 4, 5])
vit

,text,username,tweet_id,timestamp,hashtags,screen_name,retweets,likes,uni_handle,compound,Sentiment,score
0,Thanks to Developer Student Club-Vellore Insti...,Mohammad Shahid Ansari,1211639431959998464,2019-12-30 13:25:26,[],Shahid_Ansari_,0,0,0.0,0.4404,Positive,4
1,Junior Research Fellow In Vellore Institute Of...,Sarkari-Job,1211536334608904192,2019-12-30 06:35:46,[governmentjob],sarkarijobind,0,0,0.0,0.0000,Neutral,3
2,VIT Notification 2020 – Opening for Various JR...,Govt Job Guide,1210822000945418240,2019-12-28 07:17:15,[],GovtJobGuide,0,0,0.0,0.3612,Positive,4
3,"Vallore Institute of Technology (VIT) Courses,...",Rajasthan Travel and Tourism Guide,1210652629606354944,2019-12-27 20:04:14,"[rajasthanpalaces, royalrajasthan, rajasthanto...",e_rajasthan,0,0,0.0,0.0000,Neutral,3
4,MITS5509 Intelligent Systems for Analytics Ass...,Expertsmind.com,1209090557391208448,2019-12-23 12:37:07,"[MITS5509, IntelligentSystemsforAnalytics, Ass...",ExpertsMindInc,0,0,0.0,0.7825,Positive,5
5,Thanks to Developer Student Club-Vellore Insti...,Mohammad Shahid Ansari,1211639431959998464,2019-12-30 13:25:26,[],Shahid_Ansari_,0,0,0.0,0.4404,Positive,4
6,Junior Research Fellow In Vellore Institute Of...,Sarkari-Job,1211536334608904192,2019-12-30 06:35:46,[governmentjob],sarkarijobind,0,0,0.0,0.0000,Neutral,3
7,VIT Notification 2020 – Opening for Various JR...,Govt Job Guide,1210822000945418240,2019-12-28 07:17:15,[],GovtJobGuide,0,0,0.0,0.3612,Positive,4
8,"Vallore Institute of Technology (VIT) Courses,...",Rajasthan Travel and Tourism Guide,1210652629606354944,2019-12-27 20:04:14,"[rajasthanpalaces, royalrajasthan, rajasthanto...",e_rajasthan,0,0,0.0,0.0000,Neutral,3
9,MITS5509 Intelligent Systems for Analytics Ass...,Expertsmind.com,1209090557391208448,2019-12-23 12:37:07,"[MITS5509, IntelligentSystemsforAnalytics, Ass...",ExpertsMindInc,0,0,0.0,0.7825,Positive,5


In [12]:
vit.to_csv('../../Datasets/final_vit_withlikes.csv')